In [1]:
import pandas as pd
import numpy as np
import keras
import glob
import matplotlib.pyplot as plt
import scipy
from sklearn.metrics import roc_curve, auc
from keras.utils.np_utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, GlobalAveragePooling2D, Input, BatchNormalization, Multiply, Activation
from keras.optimizers import RMSprop, SGD
from keras.regularizers import l2
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import plot_model
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from keras import backend as K
import os
np.random.seed(2020)

Using TensorFlow backend.


In [2]:
#train files 전처리
train_datagen = ImageDataGenerator(featurewise_center=False,
                 samplewise_center=False,
                 featurewise_std_normalization=False,
                 samplewise_std_normalization=False,
                 zca_whitening=False,
                 rotation_range=10,
                 width_shift_range=0.05,
                 height_shift_range=0.05,
                 shear_range=0.1,
                 zoom_range=0.2,
                 channel_shift_range=0.,
                 fill_mode='nearest',
                 cval=0.,
                 horizontal_flip=True,
                 vertical_flip=False,
                 rescale=1/255,
                 validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
        "food-101\images",
        target_size=(224,224),
        batch_size=64,
        subset='training') # set as training data

#validation set 생성
validation_generator = train_datagen.flow_from_directory(
    "food-101\images",
    target_size=(224,224),
    batch_size=64,
    subset='validation') # set as validation data

Found 24800 images belonging to 31 classes.
Found 6200 images belonging to 31 classes.


In [3]:
#Modeling
model = Sequential()
model.add(Conv2D(filters = 32, kernel_size = (5,5), strides = 2, padding = 'Same', activation ='relu', input_shape = (224,224,3), kernel_initializer='he_normal'))
model.add(Conv2D(filters = 32, kernel_size = (5,5), strides = 2, padding = 'Same', activation ='relu',kernel_initializer='he_normal'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu',kernel_initializer='he_normal'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu',kernel_initializer='he_normal'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(Conv2D(filters = 128, kernel_size = (2,2),padding = 'Same', activation ='relu',kernel_initializer='he_normal'))
model.add(Conv2D(filters = 128, kernel_size = (2,2),padding = 'Same', activation ='relu',kernel_initializer='he_normal'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(Conv2D(filters = 256, kernel_size = (2,2),padding = 'Same', activation ='relu',kernel_initializer='he_normal'))
model.add(Conv2D(filters = 256, kernel_size = (2,2),padding = 'Same', activation ='relu',kernel_initializer='he_normal'))
model.add(GlobalAveragePooling2D())
model.add(Dense(512, activation = "relu",kernel_initializer='he_normal'))
model.add(Dropout(0.2))
model.add(Dense(31, activation = "softmax",kernel_initializer='he_normal',kernel_regularizer=l2()))


model.compile(optimizer = 'Adam' , loss = "categorical_crossentropy", metrics=["accuracy"])

W0105 21:14:34.604406  7816 deprecation_wrapper.py:119] From C:\Users\Owner2\Anaconda3\envs\keras2\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0105 21:14:34.620414  7816 deprecation_wrapper.py:119] From C:\Users\Owner2\Anaconda3\envs\keras2\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0105 21:14:34.622556  7816 deprecation_wrapper.py:119] From C:\Users\Owner2\Anaconda3\envs\keras2\lib\site-packages\keras\backend\tensorflow_backend.py:4185: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W0105 21:14:34.651416  7816 deprecation_wrapper.py:119] From C:\Users\Owner2\Anaconda3\envs\keras2\lib\site-packages\keras\backend\tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0105 21:14:34.6

In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 112, 112, 32)      2432      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 56, 56, 32)        25632     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 28, 28, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 28, 28, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 28, 28, 64)        18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 28, 28, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 64)        0         
__________

In [5]:
history = model.fit_generator(train_generator,steps_per_epoch=250,
                              validation_data=validation_generator,validation_steps=750/64, 
                              epochs=100)

W0105 21:14:35.046502  7816 deprecation.py:323] From C:\Users\Owner2\Anaconda3\envs\keras2\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/200
200/200 [==============================] - 216s 1s/step - loss: 3.6076 - acc: 0.0309 - val_loss: 3.4600 - val_acc: 0.0312
Epoch 2/200
200/200 [==============================] - 212s 1s/step - loss: 3.4124 - acc: 0.0449 - val_loss: 3.3397 - val_acc: 0.0547
Epoch 3/200
200/200 [==============================] - 198s 988ms/step - loss: 3.3426 - acc: 0.0560 - val_loss: 3.2963 - val_acc: 0.0664
Epoch 4/200
200/200 [==============================] - 202s 1s/step - loss: 3.2866 - acc: 0.0759 - val_loss: 3.2817 - val_acc: 0.0755
Epoch 5/200
200/200 [==============================] - 197s 983ms/step - loss: 3.2464 - acc: 0.0880 - val_loss: 3.2741 - val_acc: 0.0846
Epoch 6/200
200/200 [==============================] - 198s 989ms/step - loss: 3.2028 - acc: 0.1013 - val_loss: 3.2099 - val_acc: 0.1068
Epoch 7/200
200/200 [==============================] - 203s 1s/step - loss: 3.1735 - acc: 0.1070 - val_loss: 3.2528 - val_acc: 0.1315
Epoch 8/200
200/200 [==============================] 

200/200 [==============================] - 199s 994ms/step - loss: 1.6076 - acc: 0.5403 - val_loss: 1.7394 - val_acc: 0.5312
Epoch 62/200
200/200 [==============================] - 201s 1s/step - loss: 1.6052 - acc: 0.5408 - val_loss: 1.7889 - val_acc: 0.5312
Epoch 63/200
200/200 [==============================] - 197s 983ms/step - loss: 1.5687 - acc: 0.5518 - val_loss: 1.7541 - val_acc: 0.5234
Epoch 64/200
200/200 [==============================] - 201s 1s/step - loss: 1.5722 - acc: 0.5567 - val_loss: 1.7289 - val_acc: 0.5326
Epoch 65/200
200/200 [==============================] - 202s 1s/step - loss: 1.5494 - acc: 0.5618 - val_loss: 1.6993 - val_acc: 0.5447
Epoch 66/200
200/200 [==============================] - 200s 1s/step - loss: 1.5814 - acc: 0.5530 - val_loss: 1.9531 - val_acc: 0.4948
Epoch 67/200
200/200 [==============================] - 198s 991ms/step - loss: 1.5441 - acc: 0.5618 - val_loss: 1.8376 - val_acc: 0.4909
Epoch 68/200
200/200 [==============================] - 200

Epoch 121/200
200/200 [==============================] - 201s 1s/step - loss: 1.2301 - acc: 0.6514 - val_loss: 1.6993 - val_acc: 0.5495
Epoch 122/200
200/200 [==============================] - 203s 1s/step - loss: 1.2576 - acc: 0.6441 - val_loss: 1.5617 - val_acc: 0.5882
Epoch 123/200
200/200 [==============================] - 204s 1s/step - loss: 1.1967 - acc: 0.6565 - val_loss: 1.7460 - val_acc: 0.5339
Epoch 124/200
200/200 [==============================] - 199s 996ms/step - loss: 1.2351 - acc: 0.6452 - val_loss: 1.7178 - val_acc: 0.5495
Epoch 125/200
200/200 [==============================] - 202s 1s/step - loss: 1.1939 - acc: 0.6542 - val_loss: 1.5900 - val_acc: 0.5820
Epoch 126/200
200/200 [==============================] - 200s 998ms/step - loss: 1.2008 - acc: 0.6570 - val_loss: 1.5755 - val_acc: 0.5951
Epoch 127/200
200/200 [==============================] - 199s 993ms/step - loss: 1.1771 - acc: 0.6589 - val_loss: 1.5766 - val_acc: 0.5729
Epoch 128/200
200/200 [================

200/200 [==============================] - 201s 1s/step - loss: 1.0682 - acc: 0.6939 - val_loss: 1.6953 - val_acc: 0.5729
Epoch 181/200
200/200 [==============================] - 199s 993ms/step - loss: 1.0478 - acc: 0.6960 - val_loss: 1.7463 - val_acc: 0.5560
Epoch 182/200
200/200 [==============================] - 200s 999ms/step - loss: 1.0572 - acc: 0.6934 - val_loss: 1.5390 - val_acc: 0.6224
Epoch 183/200
200/200 [==============================] - 198s 990ms/step - loss: 1.0414 - acc: 0.6988 - val_loss: 1.6649 - val_acc: 0.5938
Epoch 184/200
200/200 [==============================] - 198s 992ms/step - loss: 1.0504 - acc: 0.6959 - val_loss: 1.6631 - val_acc: 0.5703
Epoch 185/200
200/200 [==============================] - 197s 983ms/step - loss: 1.0362 - acc: 0.6992 - val_loss: 1.6003 - val_acc: 0.5990
Epoch 186/200
200/200 [==============================] - 196s 978ms/step - loss: 1.0297 - acc: 0.7059 - val_loss: 1.6339 - val_acc: 0.5961
Epoch 187/200
200/200 [=====================

In [6]:
# 모델 평가하기
print("-- Evaluate --")

scores = model.evaluate_generator(
            validation_generator, 
            steps = 5)

print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

-- Evaluate --
acc: 58.13%


In [7]:
from keras.models import load_model

model.save('food_classfication_git_ver.h5')